In [106]:
import ROOT
import os
#import nbimporter
#import utils
import ctypes
import pandas as pd
from numpy import arange
from pprint import pprint

In [107]:
outputFolder = 'plots/'
grafana_table_csv = '../data/grafana_efficiency_table.csv'
#runs = utils.get_completed_runs_longer_than(grafana_table_csv,50)
#rad_eff_file_pattern = '/eos/cms/store/group/dpg_ctpps/comm_ctpps/pps-automation/dev/pps-test-re-tracking-efficiency/2025_test_first_bunch/{run}/outputTrackingEfficiency_run{run}.root'
track_dist_file_pattern = '/eos/cms/store/group/dpg_ctpps/comm_ctpps/pps-automation/dev/pps-test-re-tracking-efficiency/2025_test_first_bunch/{run}/outputEfficiencyAnalysisDQMHarvester_run{run}.root'

track_dist_file_first = track_dist_file_pattern.format(run=381384)

In [108]:
def fitPeak(peak_slice,fit_minX,fit_maxX):
    bin_width = peak_slice.GetXaxis().GetBinWidth(1)
    max_bin_fluctuation = 10 # 5*binwidth
    min_ndf_for_fit = 6
    max_vs_x = ROOT.TF1('max_vs_x','[0]+[1]/x+[2]/(x*x)',fit_maxX,fit_maxX)
    chi2_scan = {}
    ndf_scan = {}
    for i in range(max_bin_fluctuation+1):
        fit_minX_sl0 = fit_minX + bin_width*i
        
        for j in range(max_bin_fluctuation+1):
            fit_maxX_sl0 = fit_maxX - bin_width*j
            max_vs_x.SetRange(fit_minX_sl0,fit_maxX_sl0)
            max_vs_x.SetParameters(0,0)
            fitResults = peak_slice.Fit(max_vs_x,'SQNR')
            try:
                if fitResults.Ndf() < min_ndf_for_fit:
                    continue
            except ReferenceError as e:
                continue
            chi2_scan[(i,j)] = fitResults.Chi2()
            ndf_scan[(i,j)] = fitResults.Ndf()
    
    best_fluctuation = min(chi2_scan,key=chi2_scan.get)
    print('Best X fit range: ({minx:.2f},{maxx:.2f})'.format(
        minx=fit_minX + bin_width*best_fluctuation[0],
        maxx=fit_maxX - bin_width*best_fluctuation[1]
    ))
    print('Chi2: '+str(chi2_scan[best_fluctuation]))
    print('Ndf: '+str(ndf_scan[best_fluctuation]))
    
    
    # Do the fit after finding the best option
    fit_minX_sl0 = fit_minX + bin_width*best_fluctuation[0]
    fit_maxX_sl0 = fit_maxX - bin_width*best_fluctuation[1]
    max_vs_x.SetRange(fit_minX_sl0,fit_maxX_sl0)
    max_vs_x.SetParameters(0,0)
    _ = peak_slice.Fit(max_vs_x,'SQNR')
    return max_vs_x


In [109]:
def fitDistribution(hist,draw=None):
    """Fit the slices of the track distribution"""
    fit_minX = 0
    fit_maxX = 5
    fit_minBin = hist.GetXaxis().FindBin(fit_minX)
    fit_maxBin = hist.GetXaxis().FindBin(fit_maxX)
    
    ymax_bin = -1
    py = hist.ProjectionY()
    ymax_bin = py.GetMaximumBin()
    ymax = py.GetXaxis().GetBinCenter(ymax_bin)
    fit_yrange = py.GetRMS() / 3
    
    gaus = ROOT.TF1("gaus", "gaus", ymax-fit_yrange, ymax+fit_yrange);
    slices = ROOT.TObjArray()
    hist.FitSlicesY(gaus, fit_minBin, fit_maxBin, 0, "QNRG2",slices)

    # Now fit each parameter
    max_vs_x = fitPeak(slices[0],fit_minX,fit_maxX)
    
    beam_axis = ROOT.TF1('beam_axis','pol1',fit_minX,fit_maxX)
    _ = slices[1].Fit(beam_axis,'SQNR')
    sigma_vs_x = ROOT.TF1('sigma_vs_x','pol1',fit_minX,fit_maxX)
    _ = slices[2].Fit(sigma_vs_x,'SQNR')
    canvas=ROOT.TCanvas()
    
    if draw is not None:
        slices[draw].DrawCopy()
        if draw == 0:
            max_vs_x.Draw('same')
        elif draw == 1:
            beam_axis.Draw('same')
        elif draw == 2:
            sigma_vs_x.Draw('same')
    
    
    slices[1].SetDirectory(0)
    return slices[0],slices[1],max_vs_x,beam_axis,sigma_vs_x,canvas


In [110]:
def findXRange(peak_hist,peak_fit,target_fraction):
    min_x = ctypes.c_double()
    max_x = ctypes.c_double()
    peak_fit.GetRange(min_x,max_x)
    min_bin_x = peak_hist.GetXaxis().FindBin(min_x)
    max_bin_x = peak_hist.GetXaxis().FindBin(max_x)


    int_full = peak_hist.Integral(min_bin_x,max_bin_x)
    int_scan = {}
    for i in range(max_bin_x-min_bin_x):
        int_range = peak_hist.Integral(min_bin_x,max_bin_x-i)
        int_scan[i] = abs((int_range/int_full) - target_fraction)

    best_max_i = min(int_scan,key=int_scan.get)
    best_max_bin = max_bin_x-best_max_i
    best_max = peak_hist.GetXaxis().GetBinUpEdge(best_max_bin)
    print('Best integration range: ({minx:.2f},{maxx:.2f})'.format(minx=min_x.value,maxx=best_max))
    print('Fraction: {int_range:.3f} (target = {target:.2f})'.format(
        int_range = peak_hist.Integral(min_bin_x,best_max_bin)/int_full,
        target=target_fraction
    ))
    return min_x.value,best_max

In [111]:
import ROOT

def getTrackHist(filename, sector, station):
    """
    Restituisce l'istogramma ROOT h2TrackHitDistribution_arm{arm}_st{station}_rp3
    dal file 'filename', navigando nel path:
    DQMData/Run 999999/{arm}/Run summary/st{station}/rp3/
    """

    # mappatura dei settori: in ROOT i nomi non sono "Arm56" ma "arm0"/"arm1"
   # mappatura dei settori
    arm_dir_map  = {"45": "Arm0", "56": "Arm1"}   # directory nel ROOT
    arm_hist_map = {"45": "arm0", "56": "arm1"}   # nome istogramma

    if sector not in arm_dir_map:
        raise ValueError(f"Settore {sector} non valido (usa '45' o '56')")

    arm_dir  = arm_dir_map[sector]   # per il path
    arm_hist = arm_hist_map[sector]  # per il nome dell'istogramma
    

    # apri file
    f = ROOT.TFile.Open(filename)
    if not f or f.IsZombie():
        raise IOError(f"Impossibile aprire file {filename}")

    # costruisci il path interno
    dir_path = f"DQMData/Run 999999/{arm_dir}/Run summary/st{station}/rp3"
    hist_name = f"h2TrackHitDistribution_{arm_hist}_st{station}_rp3"

    # naviga dentro la directory
    tdir = f.Get(dir_path)
    if not tdir:
        f.Close()
        raise KeyError(f"Directory non trovata: {dir_path}")

    # prendi l'istogramma
    hist = tdir.Get(hist_name)
    if not hist:
        f.Close()
        raise KeyError(f"Istogramma non trovato: {hist_name} in {dir_path}")

    # clona per sicurezza
    hist = hist.Clone()
    hist.SetDirectory(0)

    f.Close()
    return hist



In [112]:
sector = '56'
station = '2'
n_sigma = 0.5

track_distribution_pattern = 'DQMData/Run 999999/Arm{arm}/Run summary/st{station}/rp3/h2TrackHitDistribution_arm{arm}_st{station}_rp3'
track_hist = getTrackHist(track_dist_file_first, sector, station)
#track_hist = utils.getPlot(track_dist_file_first,sector,station,track_distribution_pattern)
st_m = '220' if station == '2' else '210'
peak_points,beam_axis_points,max_vs_x,beam_axis,sigma_vs_x,canvas = fitDistribution(track_hist,0)
x_min_area,x_max_area = findXRange(peak_points,max_vs_x,0.7)

p_0_up = (
    x_min_area,
    beam_axis.Eval(x_min_area)+n_sigma*sigma_vs_x.Eval(x_min_area)
)
p_1_up = (
    x_max_area,
    beam_axis.Eval(x_max_area)+n_sigma*sigma_vs_x.Eval(x_max_area)
)
p_0_down = (
    x_min_area,
    beam_axis.Eval(x_min_area)-n_sigma*sigma_vs_x.Eval(x_min_area)
)
p_1_down = (
    x_max_area,
    beam_axis.Eval(x_max_area)-n_sigma*sigma_vs_x.Eval(x_max_area)
)

line_up = ROOT.TF1('line_up','pol1',x_min_area,x_max_area)
m_up = (p_1_up[1]-p_0_up[1])/(p_1_up[0]-p_0_up[0])
line_up.SetParameters(
    p_0_up[1]-p_0_up[0]*m_up,
    m_up
)

line_down = ROOT.TF1('line_down','pol2',x_min_area,x_max_area)
m_down = (p_1_down[1]-p_0_down[1])/(p_1_down[0]-p_0_down[0])
line_down.SetParameters(
    p_0_down[1]-p_0_down[0]*m_down,
    m_down
)

line_left = ROOT.TLine(x_min_area,line_down.Eval(x_min_area),x_min_area,line_up.Eval(x_min_area))
line_left.SetLineColor(ROOT.kRed)
line_right = ROOT.TLine(x_max_area,line_down.Eval(x_max_area),x_max_area,line_up.Eval(x_max_area))
line_right.SetLineColor(ROOT.kRed)

right_margin = 0.15
enlarge_factor = 1.5
c_hist_area = ROOT.TCanvas('c','c',round(700*enlarge_factor),round(500*enlarge_factor))
c_hist_area.SetRightMargin(right_margin)
ROOT.gStyle.SetPalette(ROOT.kViridis)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetOptTitle(0)
c_hist_area.SetLogz()

track_hist.GetXaxis().SetRangeUser(0,10)
track_hist.GetYaxis().SetRangeUser(-5,5)
track_hist.GetXaxis().SetTitleSize(0.05)
track_hist.GetXaxis().SetLabelSize(0.05)
track_hist.GetXaxis().SetTitleOffset(0.9)
track_hist.GetYaxis().SetTitleSize(0.05)
track_hist.GetYaxis().SetLabelSize(0.05)
track_hist.GetYaxis().SetTitleOffset(1)    
bin_area = track_hist.GetXaxis().GetBinWidth(1)*track_hist.GetYaxis().GetBinWidth(1)
track_hist.GetZaxis().SetTitle('Tracks/{bin_area:.3f} mm^{{2}}'.format(bin_area=bin_area))
track_hist.GetZaxis().SetTitleSize(0.04)
track_hist.GetZaxis().SetTitleOffset(1.2)
#track_hist.SetTitle('Track hit distribution - {sector}-{station}-fr'.format(sector=sector,station=st_m))

track_hist.DrawCopy('colz')
line_up.Draw('same')
line_down.Draw('same')
line_left.Draw('same')
line_right.Draw('same')

plotPubStatus = 'Preliminary'
cmsText = '#font[61]{CMS}'
cmsText2 = '#scale[0.76]{#font[52]{'+plotPubStatus+'}}'
stationTag = '#scale[1]{{#font[42]{{{sector}-{station}-fr}}}}'.format(sector=sector,station=st_m)
year_energy_tag = '#scale[0.76]{#font[42]{2024 (13.6 TeV)}}'
latex = ROOT.TLatex()
latex.SetTextSize(0.05)
latex.SetTextAlign(11)
latex.DrawLatexNDC(0.13,0.85,cmsText)
latex.DrawLatexNDC(0.13,0.80,cmsText2)
latex.DrawLatexNDC(0.13,0.13,stationTag)

latex.SetTextAlign(31)
latex.DrawLatexNDC(1-right_margin-0.01,0.91,year_energy_tag)

c_hist_area.DrawClone()
#c_hist_area.SaveAs('plots/AvgEfficiencyArea.png')
#canvas.DrawClone()

Best X fit range: (2.54,5.00)
Chi2: 832.090138418638
Ndf: 6
Best integration range: (2.54,4.17)
Fraction: 0.747 (target = 0.70)


Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in <TFitResultPtr>: TFitResult is empty - use the fit option S
Warning in <Fit>: Fit data is empty 
Error in 

In [1]:
def avgEfficiency(run, min_x, max_x, line_down, line_up, sector, station):
    # Pattern del file ROOT (globale)
    track_dist_file_pattern = '/eos/cms/store/group/dpg_ctpps/comm_ctpps/pps-automation/dev/pps-test-re-tracking-efficiency/2025_test_first_bunch/{run}/outputEfficiencyAnalysisDQMHarvester_run{run}.root'
    
    # Costruisci il percorso completo
    eff_file = track_dist_file_pattern.format(run=run)
    
    # Pattern interno del ROOT file per l'istogramma
    rad_eff_pattern = 'DQMData/Run 999999/Arm{arm}/Run summary/st{station}/rp3/h2EfficiencyMap_arm{arm}_st{station}_rp3_pl3'
    
    # Prendi l'istogramma
    hist = getTrackHist(track_dist_file_first, sector, station)
    
    # Calcolo dell'efficienza media nell'area definita
    n_bins_x = hist.GetXaxis().GetNbins()
    n_bins_y = hist.GetYaxis().GetNbins()
    eff_sum = 0
    eff_counts = 0
    for bin_x in range(1, n_bins_x + 1):
        for bin_y in range(1, n_bins_y + 1):
            x = hist.GetXaxis().GetBinCenter(bin_x)
            y = hist.GetYaxis().GetBinCenter(bin_y)
            if min_x < x < max_x and line_down.Eval(x) < y < line_up.Eval(x):
                eff_sum += hist.GetBinContent(bin_x, bin_y)
                eff_counts += 1
    
    return eff_sum / eff_counts if eff_counts > 0 else 0


In [2]:
import ROOT

# Lista delle run che vuoi analizzare
run_list = [381384, 383162, 394987]  # esempio: inserisci le tue run qui

# Parametri per il calcolo
x_min_area = 2.54  # esempio dai tuoi fit precedenti
x_max_area = 4.17
line_down = line_down  # la TF1 già definita
line_up   = line_up    # la TF1 già definita
sector = '56'
station = '2'

intLumis = []
avgEfficiencies = []

for run in run_list:
    eff_file = f'/eos/cms/store/group/dpg_ctpps/comm_ctpps/pps-automation/dev/pps-test-re-tracking-efficiency/2025_test_first_bunch/{run}/outputEfficiencyAnalysisDQMHarvester_run{run}.root'
    
    # Calcolo della luminosità integrata (qui puoi metterla manualmente o calcolarla da CSV se vuoi)
    integrated_lumi = ...  # ad esempio un float che decidi tu
    intLumis.append(integrated_lumi)
    
    # Calcolo dell'efficienza media
    avgEff = avgEfficiency(run, x_min_area, x_max_area, line_down, line_up, sector, station)
    avgEfficiencies.append(avgEff)

# Disegna il grafico
d_avg_eff = ROOT.TCanvas('c','c',1050,750)
g_avg_eff = ROOT.TGraph(len(intLumis))
for i in range(len(intLumis)):
    g_avg_eff.SetPoint(i, intLumis[i], avgEfficiencies[i])

g_avg_eff.SetTitle(';L_{{int}} (fb^{{-1}});Peak average efficiency')
g_avg_eff.SetMarkerStyle(20)
g_avg_eff.SetMarkerSize(0.5)
g_avg_eff.SetLineColor(ROOT.kBlue)
g_avg_eff.SetMarkerColor(ROOT.kAzure+2)
g_avg_eff.GetYaxis().SetRangeUser(0,1.1)
g_avg_eff.Draw('AP')
d_avg_eff.Draw()


NameError: name 'line_down' is not defined

In [ ]:
intLumis = []
avgEfficiencies = []
for run in runs:
    try:
        intLumis.append(float(pps_2023_track_df[pps_2023_track_df['run'] == run].iloc[[0]]['integrated_del_lumi']))
    except IndexError as e:
        continue
    avgEfficiencies.append(avgEfficiency(run,x_min_area,x_max_area,line_down,line_up,'56','2'))

In [ ]:
enlarge_factor = 1.5
c_avg_eff = ROOT.TCanvas('c','c',round(700*enlarge_factor),round(500*enlarge_factor))
g_avg_eff = ROOT.TGraph(len(intLumis))
for i in range(len(intLumis)):
    g_avg_eff.SetPoint(i,intLumis[i],avgEfficiencies[i])

g_avg_eff.SetTitle(';L_{{int}} (fb^{{-1}});Peak average efficiency'.format(sector=sector,station=st_m))
g_avg_eff.SetMarkerStyle(20)
g_avg_eff.SetMarkerSize(0.5)
g_avg_eff.GetYaxis().SetRangeUser(0,1.1)
g_avg_eff.SetLineColor(ROOT.kBlue)
g_avg_eff.SetMarkerColor(ROOT.kAzure+2)

g_avg_eff.GetXaxis().SetTitleSize(0.05)
g_avg_eff.GetXaxis().SetLabelSize(0.045)
g_avg_eff.GetXaxis().SetTitleOffset(0.85)
g_avg_eff.GetYaxis().SetTitleSize(0.05)
g_avg_eff.GetYaxis().SetLabelSize(0.045)
g_avg_eff.GetYaxis().SetTitleOffset(1)

plotPubStatus = 'Preliminary'
cmsText = '#font[61]{CMS} #scale[0.76]{#font[52]{'+plotPubStatus+'}}'
stationTag = '#scale[1]{{#font[42]{{{sector}-{station}-fr}}}}'.format(sector=sector,station=st_m)
year_energy_tag = '#scale[0.76]{#font[42]{2023 (13.6 TeV)}}'

latex = ROOT.TLatex()
latex.SetTextAlign(31)

c_avg_eff.SetGrid()
ROOT.gStyle.SetGridStyle(2)
ROOT.gStyle.SetGridColor(ROOT.kGray)
ROOT.gStyle.SetTitleFontSize(0.06)

g_avg_eff.Draw('AP')
latex.DrawLatexNDC(0.88,0.85,cmsText)
latex.DrawLatexNDC(0.88,0.13,stationTag)
latex.SetTextAlign(31)
latex.DrawLatexNDC(1-0.1-0.01,0.91,year_energy_tag)


lumi_first_movement = float(pps_2023_track_df[pps_2023_track_df['run'] == 367881].iloc[[0]]['integrated_del_lumi'])
lumi_second_movement = float(pps_2023_track_df[pps_2023_track_df['run'] == 369927].iloc[[0]]['integrated_del_lumi'])

line_first_movement = ROOT.TLine(lumi_first_movement,0,lumi_first_movement,1.1)
line_first_movement.SetLineColor(ROOT.kOrange-1)
line_first_movement.SetLineWidth(2)
line_first_movement.SetLineStyle(2)
line_first_movement.Draw('same')
latex.SetTextAlign(13)
latex.SetTextAngle(90)
latex.SetTextColor(ROOT.kOrange-6)
latex.DrawLatex(lumi_first_movement+0.2,0.05,'#scale[0.76]{#font[42]{Vertical movement}}')

line_second_movement = ROOT.TLine(lumi_second_movement,0,lumi_second_movement,1.1)
line_second_movement.SetLineColor(ROOT.kOrange-1)
line_second_movement.SetLineWidth(2)
line_second_movement.SetLineStyle(2)
line_second_movement.Draw('same')
latex.SetTextColor(ROOT.kOrange-6)
latex.DrawLatex(lumi_second_movement+0.2,0.05,'#scale[0.76]{#font[42]{Vertical movement}}')

c_avg_eff.Draw()
c_avg_eff.RedrawAxis()
c_avg_eff.SaveAs('plots/Eff_vs_lumi_{sector}_{station}.png'.format(sector=sector,station=st_m))